In [ ]:
import numpy as np 
import tensorflow as tf

In [ ]:
def nearest_neighbor(matrix):
    w = len(matrix)
    h = len(matrix[0])
    neighbors = [[0 for x in range(w)] for y in range(h)]
    for i in range (0, w):
        for j in range (0, h):
            entry = {
                "left": matrix[i][(j-1)%w],
                "right": matrix[i][(j+1)%h],
                "up": matrix[(i-1)%w][j],
                "down": matrix[(i+1)%w][j]
            }
            neighbors[i][j] = entry
    return neighbors        
    
    

In [ ]:
matrix = [[1,2,3,4],
         [5,6,7,8],
         [9,10,11,12],
         [13,14,15,16]]

neighbors = nearest_neighbor(matrix)

In [ ]:
def higher_dim_nearest_neighbor(tensor):
    w = len(matrix)
    h = len(matrix[0])
    d = len(matrix[0][0])
    neighbors = [[[0 for x in range(w)] for y in range(h)] for z in range d]
    for i in range (0, w):
        for j in range (0, h):
            for k in range (0,d):
                entry = [6]
                entry[0] = matrix[i][(j-1)%w][k],
                entry[1] = matrix[i][(j+1)%h][k],
                entry[2] = matrix[(i-1)%w][j][k],
                entry[3] = matrix[(i+1)%w][j][k],
                entry[4] = matrix[i][j][(k-1)%d],
                entry[5] = matrix[i][j][(k+1)%d]
            neighbors[i][j][z] = entry
    return neighbors   